In [176]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [177]:
import seaborn as sns 
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

In [178]:
df = pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [179]:
df.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [180]:
df = df.drop(["CustomerId", "Surname"], axis=1)
df = pd.get_dummies(df)

In [181]:
def categorise(df, c):
    df = df.copy()
    t1 = np.percentile(np.array(df[c]), 0)
    t2 = np.percentile(np.array(df[c]), 25)
    t3 = np.percentile(np.array(df[c]), 50)
    t4 = np.percentile(np.array(df[c]), 75)
    t5 = np.percentile(np.array(df[c]), 100)
    bins = [t1, t2, t3, t4, t5]
    new_values = pd.cut(df[c],
                        bins, 
                        labels=[1, 2, 3, 4],
                        duplicates='drop')
    return new_values

In [182]:
df.head()

,id,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0,668,33.0,3,0.00,2,1.0,0.0,181449.97,0,True,False,False,False,True
1,1,627,33.0,1,0.00,2,1.0,1.0,49503.50,0,True,False,False,False,True
2,2,678,40.0,10,0.00,2,1.0,0.0,184866.69,0,True,False,False,False,True
3,3,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0,True,False,False,False,True
4,4,716,33.0,5,0.00,2,1.0,1.0,15068.83,0,False,False,True,False,True


In [183]:
boolean_dict = {True : 1, False : 0}
cols = ["Geography_Germany", "Geography_France", "Geography_Spain", "Gender_Female", "Gender_Male"]
for col in cols:
    df[col] = df[col].map(boolean_dict)

In [184]:
df['combined_info'] = (df['NumOfProducts']+df['HasCrCard'])*df['IsActiveMember']

In [185]:
df["BalancePerProd"] = df["Balance"] / df["NumOfProducts"] 
X, y = df.drop("Exited", axis=1), df["Exited"]

In [186]:
test = pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")
test.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [187]:
test = test.drop(["CustomerId", "Surname"], axis=1)
test = pd.get_dummies(test)
test['combined_info'] = (test['NumOfProducts']+test['HasCrCard'])*test['IsActiveMember']
for col in cols:
    test[col] = test[col].map(boolean_dict)
test["BalancePerProd"] = test["Balance"] / test["NumOfProducts"] 




In [188]:
colsToScale = ["CreditScore", "Balance", "EstimatedSalary", "BalancePerProd"]
scaler = StandardScaler()
for c in colsToScale:
    X[c+"_scaled"] = scaler.fit_transform(X[[c]])
    test[c+"_scaled"] = scaler.transform(test[[c]])
X = X.drop(colsToScale, axis=1)
test = test.drop(colsToScale, axis=1)

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [190]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [191]:
score = roc_auc_score(y_test, y_pred)
print(f"The ROC area under the curve score is {score}")

The ROC area under the curve score is 0.7515882680488554


In [192]:
id = test["id"]

In [193]:

Exited = rf.predict(test)

In [194]:
submission_dict = {"id" : id, "Exited" : Exited}
submission = pd.DataFrame(data=submission_dict)

In [195]:
submission.to_csv("submission.csv", index=False)

In [196]:
import xgboost as xgb 

In [197]:
s = y_train.value_counts()
percentage = s[1] / (s[0]+s[1])

In [198]:
percentage

0.21136992767026827

In [199]:
xgb_class = xgb.XGBClassifier(num_estimators=10000, learning_rate=0.2, scale_pos_weight=int(1/percentage))
xgb_class.fit(X_train, y_train)
y_xgb = xgb_class.predict(X_test)
score_xgb = roc_auc_score(y_test, y_xgb)
print(f"The ROC area under the curve score is {score_xgb}")

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [15:44:19] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


The ROC area under the curve score is 0.8045837908537549


In [200]:
y_xgb_train = xgb_class.predict(X_train)
y_rf_train = rf.predict(X_train)

In [201]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_log = lr.predict(X_test)
score_lr = roc_auc_score(y_test, y_log)
print(f"The ROC area under the curve score is {score_lr}")

The ROC area under the curve score is 0.5


In [202]:
y_log_train = lr.predict(X_train)

In [203]:
weights = {0:percentage, 1:1-percentage}

In [204]:
Exited_xgb = xgb_class.predict(test)
submission_xgb = pd.DataFrame({"id" : id, "Exited" : Exited_xgb})
submission_xgb.to_csv("submission.csv", index=False)

In [205]:
cat = CatBoostClassifier(class_weights=weights, eval_metric='AUC', learning_rate=0.022, verbose=False)
cat.fit(X_train, y_train)
y_cat = cat.predict(X_test)
y_train_cat = cat.predict(X_train)
score_cat = roc_auc_score(y_test, y_cat)
print(f"The ROC area under the curve score is {score_cat}")

The ROC area under the curve score is 0.8056454984327215


In [206]:
Exited_cat = cat.predict(test)
submission_cat = pd.DataFrame({"id" : id, "Exited" : Exited_cat})
submission_cat.to_csv("submission.csv", index=False)